In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import sklearn

In [2]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
base = pd.read_csv("games.csv")

In [7]:
#queremos apenas os da america, dropando o que nao ira ser utilizada na rede neural
base = base.drop("Other_Sales", axis=1)
base = base.drop("Global_Sales", axis=1)
base = base.drop("Developer", axis=1)

In [9]:
base.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [11]:
#excluir as linhas com valores nulos
base = base.dropna(axis=0)

In [13]:
base.isnull().sum() #nenhum valor nulo anteriormente exlcuidos

Name               0
Platform           0
Year_of_Release    0
Genre              0
Publisher          0
NA_Sales           0
EU_Sales           0
JP_Sales           0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

In [15]:
base["Name"].value_counts()

Name
Need for Speed: Most Wanted                  8
Madden NFL 07                                8
LEGO Star Wars II: The Original Trilogy      8
The Sims 2                                   7
Terraria                                     7
                                            ..
Castlevania: Portrait of Ruin                1
Suzuki TT Superbikes                         1
Rumble Roses                                 1
Sherlock Holmes: The Mystery of the Mummy    1
STORM: Frontline Nation                      1
Name: count, Length: 4377, dtype: int64

In [17]:
base = base.drop("Name", axis=1)

In [19]:
x = base.iloc[:,[0,1,2,3,7,8,9,10,11]].values
x

array([['Wii', 2006.0, 'Sports', ..., '8', 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', ..., '8.3', 709.0, 'E'],
       ['Wii', 2009.0, 'Sports', ..., '8', 192.0, 'E'],
       ...,
       ['PC', 2014.0, 'Action', ..., '7.6', 412.0, 'M'],
       ['PC', 2011.0, 'Shooter', ..., '5.8', 43.0, 'T'],
       ['PC', 2011.0, 'Strategy', ..., '7.2', 13.0, 'E10+']], dtype=object)

In [21]:
y_na = base.iloc[:,4].values
y_eu = base.iloc[:,5].values
y_jp = base.iloc[:,6].values

In [23]:
base["Platform"].value_counts #ps2 1 0 0 0, x360 0 1 0 0 ...

<bound method IndexOpsMixin.value_counts of 0        Wii
2        Wii
3        Wii
6         DS
7        Wii
        ... 
16667    GBA
16677    GBA
16696     PC
16700     PC
16706     PC
Name: Platform, Length: 6825, dtype: object>

In [25]:
onehotencoder = ColumnTransformer(transformers=[("OneHot",OneHotEncoder(), [0,2,3,8])],remainder="passthrough")

In [27]:
x = onehotencoder.fit_transform(x).toarray()

In [29]:
x[0] #atributos que ja existiam mais as aplicacoes do one hot encoder

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 

In [31]:
#diferentemten dos processor anteriores, aqui estamos lidando com mais valores, assim sendo preferivel este modelo
camada_entrada = Input(shape=(303,))
camada_oculta1 = Dense(units=153,activation="relu")(camada_entrada) #303+3 / 2 = 153
camada_oculta2 = Dense(units=153,activation="relu")(camada_oculta1)
camada_saida1 = Dense(units=1,activation = "linear")(camada_oculta2)
camada_saida2 = Dense(units=1,activation = "linear")(camada_oculta2)
camada_saida3 = Dense(units=1,activation = "linear")(camada_oculta2)

In [33]:
regressor = Model(inputs=camada_entrada,outputs=[camada_saida1,camada_saida2,camada_saida3])

In [35]:
#problema mais complexo, utilizando o mean squared erros(mse)
regressor.compile(optimizer = "Adam", loss="mse")

In [ ]:
regressor.fit(x,[y_na,y_eu,y_jp], epochs=500, batch_size=100)

Epoch 1/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - dense_2_loss: 270.2719 - dense_3_loss: 135.5960 - dense_4_loss: 8109.2944 - loss: 8516.0000   
Epoch 2/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 2.9746 - dense_3_loss: 1.9468 - dense_4_loss: 6.0265 - loss: 10.9495
Epoch 3/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 2.4753 - dense_3_loss: 1.2098 - dense_4_loss: 0.8954 - loss: 4.5812
Epoch 4/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 4.7247 - dense_3_loss: 0.9493 - dense_4_loss: 0.6744 - loss: 6.3494
Epoch 5/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - dense_2_loss: 1.5587 - dense_3_loss: 0.9687 - dense_4_loss: 0.5079 - loss: 3.0357
Epoch 6/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.8473 - dense_3_loss: 0.6237 - dense_4_loss: 0.4436 - loss: 2.9147
Epoch 7/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - dense_2_loss: 8.2917 - dense_3_loss: 0.9146 - dense_4_loss: 0.4823 - loss: 9.6902
Epoch 8/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s